In [6]:
import os
import json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from keras.utils import to_categorical

---

In [7]:
DATASET_PATH = os.path.join("..", "..", "data", "star_types.csv")

In [8]:
df = pd.read_csv(DATASET_PATH)
df.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [9]:
# --------------------------------------------------------------------------------------
label_encoder_color = LabelEncoder()
df['Star color'] = label_encoder_color.fit_transform(df['Star color'])
label_encoder_class = LabelEncoder()
df['Spectral Class'] = label_encoder_class.fit_transform(df['Spectral Class'])

df["Temperature (K)"] = df["Temperature (K)"].astype(float)
df["Luminosity(L/Lo)"] = (df["Luminosity(L/Lo)"] * 1e5).astype(float)
df["Radius(R/Ro)"] = (df["Radius(R/Ro)"] * 1e3).astype(float)
df["Absolute magnitude(Mv)"] = (df["Absolute magnitude(Mv)"] * 1e2).astype(float)
# --------------------------------------------------------------------------------------
df["Star type"] = df["Star type"].astype(int)
# --------------------------------------------------------------------------------------
scaler = StandardScaler()
df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)"]] = scaler.fit_transform(
    df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)"]]
)
# --------------------------------------------------------------------------------------
X = df[["Temperature (K)", "Luminosity(L/Lo)", "Radius(R/Ro)", "Absolute magnitude(Mv)", "Star color", "Spectral Class"]].values
y = to_categorical(df["Star type"].values)

In [10]:
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.2, random_state=42)

train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

---

In [11]:
UPPER_BOUND_FELT252 = 3618502788666131213697322783095070105623107215331596699973092056135872020481
UPPER_BOUND__I128 = (2**127) - 1 # included

# wfloat for wsad as felt252
def wfloat_to_float(x) -> float :
    return float(
        (x - UPPER_BOUND_FELT252) if x > UPPER_BOUND__I128 \
        else x
    ) * 1e-6

# wfloat for wsad as felt252
def wfloat(x : float) :
    as_wsad = int(x*1e6)
    return (
        as_wsad + UPPER_BOUND_FELT252 if as_wsad < 0 \
        else as_wsad
    )

def to_hex(x: int) -> str:
    return f"0x{x:0x}"

def from_hex(x : str) -> int:
    return int(x, 16)

def matrix_to_wfloat(matrix) :
    return np.array([np.array([wfloat(x) for x in H]) for H in matrix])

In [12]:
print(matrix_to_wfloat(train_X[:5]))
print("--------------------")
print(matrix_to_wfloat(train_Y[:5]))

[[3618502788666131213697322783095070105623107215331596699973092056135871290719
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871561438
  946440 10000000 5000000]
 [3618502788666131213697322783095070105623107215331596699973092056135871269738
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871561053
  1127211 10000000 5000000]
 [822715
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871560966
  650546 0 1000000]
 [3618502788666131213697322783095070105623107215331596699973092056135871230609
  3618502788666131213697322783095070105623107215331596699973092056135871421858
  3618502788666131213697322783095070105623107215331596699973092056135871561173
  1243284 10000000 5000000]
 [1321220
  3618502

---

In [13]:
import basic_model_sample
from basic_model_sample import DenseLayer, Sequential, SGD

In [16]:
layers = [
    DenseLayer(input_shape=train_X.shape[1], output_shape=10, activation="ReLU"), 
    DenseLayer(output_shape=20, activation="ReLU"), 
    DenseLayer(output_shape=20, activation="ReLU"), 
    DenseLayer(output_shape=train_Y.shape[1], activation="ReLU"), 
]

network = Sequential(layers, SGD(learning_rate=0.1))
network.build()

# print(f"Num params: {network.num_params()}")

network.train(train_X, train_Y, epochs=100, batch_size=1, verbose=True)

predictions = network.forward(train_X)
print(f"prediction \n{predictions}")
loss = network.mse_loss(predictions, train_Y)
print(f"Final loss on test set: {loss}")

Epoch 1, Loss: 0.15615050023495292
Epoch 11, Loss: 0.1560451009272055
Epoch 21, Loss: 0.15310388833935504
Epoch 31, Loss: 0.1522220747393773
Epoch 41, Loss: 0.14957354623967556
Epoch 51, Loss: 0.15109561928604
Epoch 61, Loss: 0.15240218785469084
Epoch 71, Loss: 0.15122274583688533
Epoch 81, Loss: 0.1552016129660332
Epoch 91, Loss: 0.15273744458883523
prediction 
[[0.         0.09421744 0.         0.13965895 0.16155068 0.27038954]
 [0.         0.09421744 0.         0.13965895 0.16155068 0.27038954]
 [0.         0.09421744 0.         0.13965895 0.16155068 0.27038954]
 ...
 [0.         0.09421744 0.         0.13965895 0.16155068 0.27038954]
 [0.         0.09421744 0.         0.13965895 0.16155068 0.27038954]
 [0.         0.09421744 0.         0.13965895 0.16155068 0.27038954]]
Final loss on test set: 0.15218667415314452
